## Extract eyes open and eyes closed and stop timestamps to crop data for resting state functional connectivity analysis
##### Input: *_preprocessed-raw.fif
#####  Output: *_RestingTStamps.csv
######   * = Subject e.g. 018 or C14 etc

In [3]:
import csv
import os
import sys
import mne
from mne.io import read_raw_fif
#%conda install utils
#import something to save in pickle

In [9]:
annotation_keys = {'eyes closed': 1, 'Trigger#1': 1, 'EYES CLOSED': 1, # eyes closed
                      'eyes open': 2, 'eyes opened': 2, 'Trigger#2': 2, 'EYES OPEN': 2, 'eyes openned':2, # eyes open
                      'stop': 9, 'Stop': 9, 'STOP': 9, 'Sop': 9, # stop
                      }

In [8]:
# Settings
# Read in processed raw data
#data_path = '../../Data/Processed Data/'
#The data path below is local to sbflpc002
data_path = '/run/user/1000/gvfs/smb-share:server=research-cifs.nyumc.org,share=research/jwanglab/jwanglabspace/George Kenefati/Chronic Low Back Pain Study/Data/Processed Data'
save_path = '../../../Data/Eyes Timestamps/'

if not os.path.exists(save_path):
    os.makedirs(save_path)

In [10]:
file_list = os.listdir(data_path)
# Only select files that end in "raw.fif"
raw_fif_files = [file for file in file_list if file.endswith('raw.fif')]

In [5]:
# Manually change index to identify timestamps of eyes closed/open
# Manually checking bc little standardization
file = raw_fif_files[0]
file_path = os.path.join(data_path, file)
raw = read_raw_fif(file_path, preload = False)

events_from_annot, event_dict = mne.events_from_annotations(raw, event_id = annotation_keys)
# Extract timestamps from the events
timestamps_with_id = [(event[0] / raw.info['sfreq'], event[2]) for event in events_from_annot]

# Get name of the file we're looking at
#print(file_path)

# Output timestamps with associated event IDs
print("Timestamps and Associated Event IDs:")
for timestamp, event_id in timestamps_with_id:
    print(f"Timestamp: {timestamp:.3f} seconds, Event ID: {event_id}")

Opening raw data file /run/user/1000/gvfs/smb-share:server=research-cifs.nyumc.org,share=research/jwanglab/jwanglabspace/George Kenefati/Chronic Low Back Pain Study/Data/Processed Data/039_preprocessed-raw.fif...
    Range : 0 ... 602879 =      0.000 ...  1507.197 secs
Ready.
Used Annotations descriptions: ['Stop', 'eyes closed', 'eyes opened']
Timestamps and Associated Event IDs:
Timestamp: 5.223 seconds, Event ID: 1
Timestamp: 309.095 seconds, Event ID: 9
Timestamp: 339.295 seconds, Event ID: 2
Timestamp: 647.395 seconds, Event ID: 9
Timestamp: 1504.095 seconds, Event ID: 9


##### Next, collect the timepoints for the eyes closed, eyes closed stop, eyes open, and eyes open stop

In [6]:
### Script for a normal timestamp
saved_times = [None, None, None, None]  # Initialize an array to store timestamps

for idx, (timestamp, event_id) in enumerate(timestamps_with_id):
    # Find first instance of event ID 1, then the first instance of event ID 9 afterwards
    # Find first instance of event ID 2, the the first instance of event ID 9 afterwards
    if event_id == 1:
        if saved_times[0] is None:  # Check if the first index is empty
            saved_times[0] = timestamp
    elif event_id == 9 and idx > 0 and timestamps_with_id[idx - 1][1] == 1:
        saved_times[1] = timestamp
    elif event_id == 2:
        if saved_times[2] is None:  # Check if the third index is empty
            saved_times[2] = timestamp
    elif event_id == 9 and idx > 0 and timestamps_with_id[idx - 1][1] == 2:
        saved_times[3] = timestamp

print("Saved Times:", saved_times)

Saved Times: [5.2225, 309.095, 339.295, 647.395]


In [16]:
# Function to extract subject ID from the file path
def extract_subject_id(file_path):
    file_name = file_path.split('/')[-1]  # Extract the file name
    subject_id = file_name.split('_')[0]  # Get the part before '_preprocessed-raw.fif'
    return subject_id

# Extract subject ID from the given file paths
subject_id = extract_subject_id(file_path)

# Define the CSV file name
csv_file_name = f"{subject_id}_RestingTStamps.csv"

# Construct the full file path
full_file_path = os.path.join(save_path, csv_file_name)

# Write data to the CSV file
with open(full_file_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Seconds"])

    for timestamp in saved_times:
        writer.writerow([timestamp])
    

In [19]:
## SCRIPT START FOR IRREGULAR TIMES/ANNOTATIONS
"""  #USE IF FILE DOES NOT COMPLY WITH PARSING 
from datetime import datetime

file = raw_fif_files[43]
file_path = os.path.join(data_path, file)
raw = read_raw_fif(file_path, preload = False)

events_from_annot, event_dict = mne.events_from_annotations(raw) #Removed annotations_key to see all annotations

# Extract timestamps from the events
timestamps_with_id = [(event[0] / raw.info['sfreq'], event[2]) for event in events_from_annot]

timestamps_event_ids_annotations = []
for idx, event in enumerate(events_from_annot):
    timestamp = event[0] / raw.info['sfreq']  # Timestamp in seconds
    event_id = event[2]  # Event ID
    #use event_dict to get the annotation
    timestamps_event_ids_annotations.append((timestamp, event_id))

# Print the extracted data
for timestamp, event_id in timestamps_event_ids_annotations:
    print(f"Timestamp: {timestamp:.3f}s, Event ID: {event_id}")

"""

Opening raw data file /run/user/1000/gvfs/smb-share:server=research-cifs.nyumc.org,share=research/jwanglab/jwanglabspace/George Kenefati/Chronic Low Back Pain Study/Data/Processed Data/008_preprocessed-raw.fif...
    Range : 0 ... 712799 =      0.000 ...  1781.997 secs
Ready.


TypeError: 'datetime.datetime' object cannot be interpreted as an integer

Channels marked as bad:
none


In [13]:
event_dict

{'Trigger#1': 1,
 'Trigger#2': 2,
 'Trigger#3': 3,
 'Trigger#4': 4,
 'Trigger#6': 5}

In [14]:
raw.plot()

Using qt as 2D backend.


In [15]:
## SCRIPT CONTINUED FOR IRREGULAR TIMES/ANNOTATIONS
saved_times = [None, None, None, None]  # Initialize an array to store timestamps
# Have manual adjustments to saved times
saved_times[0] = 590.695
saved_times[1] = 628.705 - 30
saved_times[2] = 290.420
saved_times[3] = 590.695 - 30
print("Saved Times:", saved_times)

Saved Times: [590.695, 598.705, 290.42, 560.695]


## Script below should be used after all CSVs are created

In [17]:
# Find the length of the recordings
import os
import pandas as pd
from datetime import timedelta

def process_csv_files(csv_path):
    # List all CSV files in the given path
    csv_files = [f for f in os.listdir(csv_path) if f.endswith('.csv')]

    for file_name in csv_files:
        file_path = os.path.join(csv_path, file_name)
        
        # Read the CSV file into a DataFrame
        df = pd.read_csv(file_path)
        
        # Extract values from the DataFrame
        EC_start = df['Seconds'][0]
        EC_stop = df['Seconds'][1]
        EO_start = df['Seconds'][2]
        EO_stop = df['Seconds'][3]
        
        # Calculate time differences
        time_diff_EC = timedelta(seconds=EC_stop - EC_start)
        time_diff_EO = timedelta(seconds=EO_stop - EO_start)
        
        # Convert time differences to minutes and seconds
        minutes_EC, seconds_EC = divmod(time_diff_EC.total_seconds(), 60)
        minutes_EO, seconds_EO = divmod(time_diff_EO.total_seconds(), 60)
        
        print(f"File: {file_name}")
        print(f"Length of time between EC_start and EC_stop: {int(minutes_EC)} minutes and {int(seconds_EC)} seconds")
        print(f"Length of time between EO_start and EO_stop: {int(minutes_EO)} minutes and {int(seconds_EO)} seconds")
        print("=============================")

# Use function defined above
csv_path = '../../../Data/Eyes Timestamps/'

process_csv_files(csv_path)


File: 037_RestingTStamps.csv
Length of time between EC_start and EC_stop: 5 minutes and 12 seconds
Length of time between EO_start and EO_stop: 5 minutes and 13 seconds
File: 034_RestingTStamps.csv
Length of time between EC_start and EC_stop: 5 minutes and 3 seconds
Length of time between EO_start and EO_stop: 5 minutes and 1 seconds
File: 049_RestingTStamps.csv
Length of time between EC_start and EC_stop: 5 minutes and 17 seconds
Length of time between EO_start and EO_stop: 5 minutes and 4 seconds
File: 056_RestingTStamps.csv
Length of time between EC_start and EC_stop: 4 minutes and 57 seconds
Length of time between EO_start and EO_stop: 5 minutes and 1 seconds
File: 021_RestingTStamps.csv
Length of time between EC_start and EC_stop: 5 minutes and 14 seconds
Length of time between EO_start and EO_stop: 4 minutes and 53 seconds
File: 014_RestingTStamps.csv
Length of time between EC_start and EC_stop: 0 minutes and 8 seconds
Length of time between EO_start and EO_stop: 4 minutes and 30